In [1]:
%load_ext iminizinc

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display
import IPython
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

from datetime import datetime
# import talib
import pandas_ta as ta
# from talib.abstract import *
from math import *
from collections import OrderedDict
# import vectorbt as vbt    
import json
from decimal import *
getcontext().prec = 6
import seaborn as sns

# import mplfinance as mpf
%matplotlib inline

mpl.rcParams.update({'font.size': 3, 'lines.linewidth': 0.5, 'figure.dpi': 300})
plt.rcParams['lines.linewidth'] = 0.5 



<IPython.core.display.Javascript object>

MiniZinc to FlatZinc converter, version 2.7.5, build 891740483
Copyright (C) 2014-2023 Monash University, NICTA, Data61


In [283]:

def plot_candles(wdf, ax=None, kwargs={}):
    if ax is None:
        fig, ax = plt.subplots(**kwargs)

    up, down = wdf[wdf.close >= wdf.open], wdf[wdf.close < wdf.open]
    col1,col2 = 'green','red'
    width, width2 = .1, .02
    # Plotting up prices of the stock
    ax.bar(up.index, up.close-up.open, width, bottom=up.open, color=col1)
    ax.bar(up.index, up.high-up.close, width2, bottom=up.close, color=col1)
    ax.bar(up.index, up.low-up.open, width2, bottom=up.open, color=col1)
    # Plotting down prices of the stock
    ax.bar(down.index, down.close-down.open, width, bottom=down.open, color=col2)
    ax.bar(down.index, down.high-down.open, width2, bottom=down.open, color=col2)
    ax.bar(down.index, down.low-down.close, width2, bottom=down.close, color=col2)    
    plt.xticks(rotation=30, ha='right')
    
def load_candles(pair, timeframe):
    odf = pd.read_json(f'../freq-user-data/data/binance/{pair}-{timeframe}.json').dropna()
    odf.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']

    odf['date'] = pd.to_datetime(odf['timestamp'], unit='ms', utc=False)
    # df.index = df['time']
    # df.set_index('time', drop=True, inplace=True)
    odf['idate'] = odf.date.dt.strftime('%Y%m%d')
    odf.set_index(pd.DatetimeIndex(odf["date"]), inplace=True, drop=True)
    # df = df[['time', 'symbol', 'source', 'resolution', 'open', 'high', 'low', 'close', 'volume']]
    # df.to_csv (r'./data/binance/BTC_USDT-5m.csv', index = None)
    # df.set_index('time')
    odf = odf.sort_index()
    return odf



def add_indicators(wdf, hold, lag, ):
    wdf['rfd'] = wdf.low.rolling(hold).min().shift(-hold)
    wdf['rfu'] = wdf.high.rolling(hold).max().shift(-hold)
    wdf['lr_fu'] = np.log(wdf.rfu / wdf.close)
    wdf['lr_fd'] = np.log(wdf.rfd / wdf.close)
    
    wdf=wdf.join(wdf[['lr_fu', 'lr_fd']].apply(lambda x: x.multiply(100).round(2)).add_prefix('p'))
    # wdf=wdf.join(wdf[['plr_fu', 'plr_fd', 'plr_wiu', 'plr_wid']].apply(lambda x: pd.qcut(x,ncuts)).add_prefix('q'))

    # wdf['qmprfu'] = pd.qcut(wdf.mprfu, 10)
    return wdf

def call_mzn_spans(sdf,lag_n):
    # 
    # bags = {f'g{key}': list(map(int, sorted(sdf[key].values.nonzero()[0] + 1))) for key in ['ba_sig', 'ba_is_beg', 'ba_is_end']}
    bags = {f'g{key}': list(map(int, sorted(sdf[key].values.nonzero()[0] + 1))) for key in ['ba_sig', 'ba_is_beg', 'ba_is_end']}
    model_params = {
        'lag_n': lag_n, #'ba_sig': sdf.ba_sig.values.tolist(), 'ba_in': sdf.ba_in.values.tolist()
         **{ key: sdf[key].values.tolist() for key in ['ba_in']},
         **{ key: bags[key] for key in bags}
    }
    with open('spans.dzn.json', 'w+') as f: f.write(json.dumps(model_params, indent=2))

    # print('Model params:', model_params)

    # mzn_model = Model("./ranges.mzn")
    # gecode = Solver.lookup("gecode")
    # instance = Instance(gecode, mzn_model)

    # for k,v in model_params.items(): instance[k] = v
    # result = instance.solve()

    # return result

def make_spans(wdf, lag_n, max_rat):
    sdf = pd.DataFrame({
        'ba_max' : wdf.close.rolling(lag_n).max(),
        'ba_min' : wdf.close.rolling(lag_n).min(),
    }).fillna(method='bfill'
    ).assign(ba_rat = lambda df:df['ba_max']/df['ba_min'] - 1
    ).assign(ba_in=lambda x:(x['ba_rat'] <= max_rat) 
    ).assign(ba_sig=lambda x: x['ba_in'] & (~x['ba_in']).shift()
    ).assign(ba_is_beg=lambda x:x['ba_sig'].shift(-(lag_n-1)).fillna(False)
    ).assign(ba_lag=lambda x: x['ba_is_beg'].rolling(lag_n).max().fillna(0).astype(bool)
    ).assign(ba_is_end=lambda x: x['ba_in'] & (~x['ba_in']).shift(-1))
    sdf.at[sdf.index[-1] , 'ba_is_end'] = sdf.iloc[-1].ba_in


    i_begends = np.array([sorted(sdf[key].values.nonzero()[0] + 1) for key in ['ba_is_beg', 'ba_is_end']])
    spa_min_max = np.array([(x.min(),x.max()) for x in [wdf.iloc[r[0]:r[1]].close for r in i_begends.T]]).T
    # ix_spans = [wdf.index[iba_begs[i]:iba_ends[i]+1] for i in range(len(iba_begs))]

    sdf.to_csv('test.csv')
    # call_mzn_spans(sdf, lag_n)
    
    # ba_is_beg, ba_is_end = [(map(int, sorted(sdf[key].values.nonzero()[0] + 1))) for key in ['ba_is_beg', 'ba_is_end']]

    # spa_mins, spa_maxes = wdf.close.values[ba_is_beg].min(), wdf.close.values[ba_is_end].min()
    return sdf, i_begends, spa_min_max

lag_n, max_rat = 5, 0.015
sdf, i_begends = make_spans(wdf, lag_n, max_rat)


array([[10124.98, 10221.76,  9858.54,  9609.94,  9702.27, 10059.3 ,
         9551.96,  9590.9 ,  9825.57,  9520.16,  8739.68,  8804.36,
         8650.  ,  8531.88,  8700.  ,  9001.4 ,  6153.9 ,  6368.68,
         6696.19,  6772.78,  7287.55,  7276.83,  6831.28,  6803.22,
         6699.34,  6817.77,  7027.55,  7146.77,  7097.33,  6793.29,
         7061.54,  7480.78,  7675.58,  8727.82],
       [10269.24, 10344.36, 10000.58,  9724.82,  9830.68, 10164.71,
         9607.23,  9722.56,  9936.4 ,  9656.13,  8847.16,  8915.34,
         8745.81,  8648.37,  8816.27,  9150.9 ,  6236.78,  6463.47,
         6768.15,  6843.82,  7381.01,  7361.28,  6907.82,  6878.21,
         6772.74,  6918.18,  7104.52,  7248.6 ,  7184.94,  6878.68,
         7133.39,  7568.68,  7815.3 ,  8834.08]])

In [3]:
odf = load_candles('BTC_USDT', '4h')

In [11]:
from minizinc import Instance, Model, Solver
import nest_asyncio
nest_asyncio.apply()

In [ ]:

    # with open('all-swing_highs-result.json', 'w+') as f: f.write(json.dumps({'lha': result['lha'], 'nsn': result['nsn']}))


    # with open('all-swing_highs-result.json') as f: result = json.loads(f.read())

    # lha, nsn = resarr = [result['lha'],result['nsn']]
    # print(f'nsn={nsn}\nlha=\n{lha[0]}\n{lha[1]}')


    # idxs = (np.array(lha) - 1).T.tolist()
    # ##############################################################

In [174]:

### Ranges
from matplotlib import pyplot as plt

strategy = 'stats-ranges'
strategy_params_json = f'./{strategy}.json'

mpl.rcParams.update({'font.size': 3, 'lines.linewidth': 0.5, 'figure.dpi': 300})

dlen = len(odf)


sl_n2= IntSlider(description="n2", min=0, max=ceil(log(dlen, 2.0)+1), step=1, value=9)
sl_w = IntSlider(description="w", min=0, max=ceil(dlen/pow(2,sl_n2.value-1)-1), step=1, value=1)
def update_sl_w_range(*args):
    sl_w.max = ceil(dlen/pow(2,sl_n2.value-1)-1)
sl_n2.observe(update_sl_w_range, 'value')

sl_hold = IntSlider(value=7,min=1,max=50,step=1,description='Hold',continuous_update=False)
sl_lag_n = IntSlider(value=5,min=1,max=50,step=1,description='lag_n',continuous_update=False)
tx_max_rat = FloatText(value=0.05,min=0.01,max=0.1,step=0.001,description='max_rat',continuous_update=False)
chk_candles = Checkbox(value=True, description='Candles', disabled=False)
dd_dir = Dropdown(options=[('Up', 1), ('Down', -1)], value=1, description='Dir',)

strategy_params = {
    "hold": {'wdg': sl_hold},
    "candles": {"wdg": chk_candles},
    "mdir": {"wdg": dd_dir},
    "lag_n": {"wdg": sl_lag_n},
    "max_rat": {"wdg": tx_max_rat}
}
wdgts = [pv['wdg'] for pk, pv in strategy_params.items()]

ui = widgets.VBox([
    widgets.HBox([sl_n2, sl_w]),
    widgets.VBox([widgets.HBox(wdgts[i:i+4]) for i in range(0, len(wdgts), 4)])
])

if os.path.exists(strategy_params_json):
    with open(strategy_params_json) as f: 
        js = json.loads(f.read());
        for k, v in strategy_params.items(): 
            if k in js: v['wdg'].value=js[k];

else: print(f'File not found: {strategy_params_json}')

wdf = None
result = None

def printer(
        w2log, w, hold, candles, mdir, lag_n, max_rat
):
    global wdf
    global result
    with open(strategy_params_json, "w") as f: f.write(json.dumps({k: v['wdg'].value for k, v in strategy_params.items()}))
    
    
    wsz = floor(pow(2,w2log))
    wst = floor(w * wsz / 2)
    wed = floor(wst + wsz)
    
    wdf = odf.iloc[wst:wed,:].copy()    
    wlen = len(wdf)

    sdf, ba_is_beg, ba_is_end = make_spans(wdf, lag_n, max_rat)
    # spa_beg, spa_end = [wdf.index[x] for x in [ba_is_beg, ba_is_end]]
    idx_spa_beg, idx_spa_end = wdf.index[ba_is_beg], wdf.index[ba_is_end]

    print(f'N={len(wdf)}, Period: {wdf.index[-1] - wdf.index[0]}, Start: {wdf.index[0]}, End: {wdf.index[-1]}\n')

    plt.close('all')
    fig = plt.figure(figsize=(9,3))

    (ax1,ax2, ax3) = fig.subplots(3, 1, height_ratios=[3,1,1], sharex=True);
    plot_candles(wdf, ax=ax1) if candles else ax1.plot(wdf.close, lw=0.3, c='b')

out = widgets.interactive_output(printer, {
        "w2log":sl_n2, "w":sl_w, 
        **{k : v['wdg'] for k,v in strategy_params.items()}
    });
x = display(ui, out);


Output()